In [5]:
pip install faker


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sqlite3
import uuid
from faker import Faker
import random
from datetime import datetime, timedelta

# Initialize Faker and SQLite connection
faker = Faker()
conn = sqlite3.connect('banking.db')
cursor = conn.cursor()

# Create Tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS customers (
    id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    dob DATE,
    email TEXT,
    phone TEXT,
    address TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS branches (
    branch_id INTEGER PRIMARY KEY,
    branch_name TEXT,
    location TEXT,
    opening_hours TEXT,
    is_open BOOLEAN
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS accounts (
    account_id TEXT PRIMARY KEY,
    customer_id INTEGER,
    branch_id INTEGER,
    balance REAL,
    account_type TEXT,
    opened_on DATE,
    status TEXT,
    FOREIGN KEY (customer_id) REFERENCES customers(id),
    FOREIGN KEY (branch_id) REFERENCES branches(branch_id)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS transactions (
    transaction_id TEXT PRIMARY KEY,
    account_id TEXT,
    amount REAL,
    transaction_type TEXT,
    transaction_date DATE,
    status TEXT,
    description TEXT,
    FOREIGN KEY (account_id) REFERENCES accounts(account_id)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS loans (
    loan_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    loan_amount REAL,
    interest_rate REAL,
    loan_type TEXT,
    approved_on DATE,
    due_date DATE,
    status TEXT,
    FOREIGN KEY (customer_id) REFERENCES customers(id)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS cards (
    card_id TEXT PRIMARY KEY,
    account_id TEXT,
    card_type TEXT,
    expiry_date DATE,
    status TEXT,
    credit_limit REAL,  -- Renamed 'limit' to 'credit_limit'
    FOREIGN KEY (account_id) REFERENCES accounts(account_id)
)
''')

# Insert Random Data into Branches
branches = ['Main', 'West End', 'Downtown', 'Eastside']
for branch_name in branches:
    cursor.execute('''
    INSERT INTO branches (branch_name, location, opening_hours, is_open) 
    VALUES (?, ?, ?, ?)
    ''', (branch_name, faker.address(), "9 AM - 5 PM", random.choice([True, False])))

# Insert Random Data into Customers
for _ in range(1000):
    cursor.execute('''
    INSERT INTO customers (first_name, last_name, dob, email, phone, address) 
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (faker.first_name(), faker.last_name(), faker.date_of_birth(minimum_age=18, maximum_age=90), 
          faker.email(), faker.phone_number(), faker.address()))

# Insert Random Data into Accounts
for _ in range(1000):
    customer_id = random.randint(1, 1000)
    branch_id = random.randint(1, 4)
    cursor.execute('''
    INSERT INTO accounts (account_id, customer_id, branch_id, balance, account_type, opened_on, status) 
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (str(uuid.uuid4()), customer_id, branch_id, round(random.uniform(1000.0, 100000.0), 2), 
          random.choice(['Checking', 'Savings']), faker.date_this_decade(), random.choice(['Active', 'Inactive'])))

# Insert Random Data into Transactions
for _ in range(2000):
    account_id = str(uuid.uuid4())  # Ensure valid UUID
    cursor.execute('''
    INSERT INTO transactions (transaction_id, account_id, amount, transaction_type, transaction_date, status, description) 
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (str(uuid.uuid4()), account_id, round(random.uniform(50.0, 5000.0), 2), 
          random.choice(['Deposit', 'Withdrawal', 'Transfer']), 
          faker.date_this_year(), random.choice(['Success', 'Failed']), faker.text()))

# Insert Random Data into Loans
for _ in range(300):
    customer_id = random.randint(1, 1000)
    cursor.execute('''
    INSERT INTO loans (customer_id, loan_amount, interest_rate, loan_type, approved_on, due_date, status) 
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (customer_id, round(random.uniform(5000.0, 100000.0), 2), 
          round(random.uniform(2.5, 10.0), 2), random.choice(['Home', 'Auto', 'Personal']),
          faker.date_this_decade(), faker.date_this_decade(), random.choice(['Approved', 'Pending', 'Rejected'])))

# Insert Random Data into Cards
for _ in range(1000):
    account_id = str(uuid.uuid4())
    cursor.execute('''
    INSERT INTO cards (card_id, account_id, card_type, expiry_date, status, credit_limit) 
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (str(uuid.uuid4()), account_id, random.choice(['Debit', 'Credit']), faker.date_this_century(),
          random.choice(['Active', 'Blocked']), round(random.uniform(500.0, 50000.0), 2)))

# Commit the changes
conn.commit()

# Close the connection
conn.close()
